##### データセットの場所やバッチサイズなどの定数値の設定

In [3]:
import os
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'


# 使用するデバイス
# GPU を使用しない環境（CPU環境）で実行する場合は DEVICE = 'cpu' とする
DEVICE = 'cuda:0'

# 全ての訓練データを一回ずつ使用することを「1エポック」として，何エポック分学習するか
# 再開モードの場合も, このエポック数の分だけ追加学習される（N_EPOCHSは最終エポック番号ではない）
N_EPOCHS = 100

# 学習時のバッチサイズ
BATCH_SIZE = 64

# 訓練データセット（画像ファイルリスト）のファイル名
DATASET_CSV = './tinyCelebA/image_list.csv'

# 画像ファイルの先頭に付加する文字列（データセットが存在するディレクトリのパス）
DATA_DIR = './tinyCelebA/'

# 画像サイズ
H = 128 # 縦幅
W = 128 # 横幅
C = 3 # チャンネル数（カラー画像なら3，グレースケール画像なら1）

# 特徴ベクトルの次元数
N = 128

# 学習結果の保存先フォルダ
MODEL_DIR = './GAN_models/'

# 学習結果のニューラルネットワークの保存先
MODEL_FILE_G = os.path.join(MODEL_DIR, './face_generator_model.pth') # ジェネレータ
MODEL_FILE_D = os.path.join(MODEL_DIR, './face_discriminator_model.pth') # ディスクリミネータ

# 中断／再開の際に用いる一時ファイル
CHECKPOINT_EPOCH = os.path.join(MODEL_DIR, 'checkpoint_epoch.pkl')
CHECKPOINT_GEN_MODEL = os.path.join(MODEL_DIR, 'checkpoint_gen_model.pth')
CHECKPOINT_DIS_MODEL = os.path.join(MODEL_DIR, 'checkpoint_dis_model.pth')
CHECKPOINT_GEN_OPT = os.path.join(MODEL_DIR, 'checkpoint_gen_opt.pth')
CHECKPOINT_DIS_OPT = os.path.join(MODEL_DIR, 'checkpoint_dis_opt.pth')

##### ニューラルネットワークモデルの定義

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from mylib.basic_layers import Reshape, MinibatchDiscrimination, DiscriminatorAugmentation


# Pre-act Residual Block
# 通常の Residual Block では畳込みの後に活性化関数をかけるのに対し，その順序を逆にして先に活性化関数をかけるようにしたもの
# なお，ここではバッチ正規化の代わりに spectral normalization という正規化手法を選択できるようにしている（sn=Trueを指定すると spectral normalization を使用できる）
# spectral normalization は GAN の学習の安定化に有効で，基本的にはディスクリミネータで使用する
# 参考サイト: https://qiita.com/SZZZUJg97M/items/371f694f05998439bd45 など
class ResBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding, sn=False):
        super(ResBlock, self).__init__()
        shortcut_conv = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride, padding=padding)
        main_conv1 = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride, padding=padding)
        main_conv2 = nn.Conv2d(in_channels=out_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride, padding=padding)
        if sn:
            # spectral normalization を用いる場合（主にディスクリミネータ用）
            self.shortcut = nn.utils.spectral_norm(shortcut_conv) 
            self.block1 = nn.Sequential(nn.ReLU(), nn.utils.spectral_norm(main_conv1))
            self.block2 = nn.Sequential(nn.ReLU(), nn.utils.spectral_norm(main_conv2))
        else:
            # バッチ正規化を用いる場合（主にジェネレータ用）
            self.shortcut = shortcut_conv
            self.block1 = nn.Sequential(nn.BatchNorm2d(num_features=in_channels), nn.ReLU(), main_conv1)
            self.block2 = nn.Sequential(nn.BatchNorm2d(num_features=out_channels), nn.ReLU(), main_conv2)
    def forward(self, x):
        s = self.shortcut(x)
        h = self.block1(x)
        h = self.block2(h)
        return h + s


# GANジェネレータ用のアップサンプリング層
# 最近傍補間で特徴マップの縦幅・横幅を 2 倍に拡大したのち，Residual Block を適用する
# この方法は逆畳込み（nn.ConvTranspose2d）の代わりとして使用でき，かつ，逆畳込みより checker board artifact が生じにくいと言われている
class myUpsamplingBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(myUpsamplingBlock, self).__init__()
        self.up = nn.UpsamplingNearest2d(scale_factor=2)
        self.rb = ResBlock(in_channels=in_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=1, sn=False)
    def forward(self, x):
        h = self.up(x)
        return self.rb(h)


# GANディスクリミネータ用のダウンサンプリング層
# Residual Block を適用したのち，average pooling を実行することにより特徴マップの縦幅・横幅を 1/2 に縮小する
class myDownsamplingBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(myDownsamplingBlock, self).__init__()
        self.rb = ResBlock(in_channels=in_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=1, sn=True)
        self.down = nn.AvgPool2d(kernel_size=2)
    def forward(self, x):
        h = self.rb(x)
        return self.down(h)


# 顔画像生成ニューラルネットワーク
# GAN生成器（ジェネレータ）のサンプル
class Generator(nn.Module):

    # C: 出力顔画像のチャンネル数（1または3と仮定）
    # H: 出力顔画像の縦幅（32の倍数と仮定）
    # W: 出力顔画像の横幅（32の倍数と仮定）
    # N: 入力の特徴ベクトル（乱数ベクトル）の次元数
    def __init__(self, C, H, W, N):
        super(Generator, self).__init__()

        # 前処理のための層
        # 入力の特徴ベクトルを，チャンネル数 512, 縦幅 H/32, 横幅 W/32 の特徴マップに変換するために使用
        self.conv0 = nn.Sequential(
            Reshape(size=(N, 1, 1)),
            nn.ConvTranspose2d(in_channels=N, out_channels=512, kernel_size=(H//32, W//32), stride=1, padding=0),
        )

        # アップサンプリング層1～5
        # これらを通すことにより特徴マップの縦幅・横幅がそれぞれ 2 倍になる
        # 5つ通すことになるので，最終的には都合 32 倍になる -> ゆえに縦幅 H/32, 横幅 W/32 の特徴マップからスタートする
        self.up1 = myUpsamplingBlock(in_channels=512, out_channels=256)
        self.up2 = myUpsamplingBlock(in_channels=256, out_channels=128)
        self.up3 = myUpsamplingBlock(in_channels=128, out_channels=64)
        self.up4 = myUpsamplingBlock(in_channels=64, out_channels=32)
        self.up5 = myUpsamplingBlock(in_channels=32, out_channels=32)

        # 出力画像生成用の畳込み層
        self.conv5 = nn.Sequential(
            nn.BatchNorm2d(num_features=32),
            nn.ReLU(),
            nn.Conv2d(in_channels=32, out_channels=C, kernel_size=1, stride=1, padding=0),
        )

    def forward(self, z):
        h = self.conv0(z) # N 次元の特徴ベクトルをチャンネル数 512, 縦幅 H/32, 横幅 W/32 の特徴マップに変換
        h = self.up1(h)
        h = self.up2(h)
        h = self.up3(h)
        h = self.up4(h)
        h = self.up5(h)
        y = torch.tanh(self.conv5(h))
        return y


# 顔画像が Real か Fake を判定するニューラルネットワーク
# GAN識別器（ディスクリミネータ）のサンプル
class Discriminator(nn.Module):

    # C: 入力顔画像のチャンネル数（1または3と仮定）
    # H: 入力顔画像の縦幅（32の倍数と仮定）
    # W: 入力顔画像の横幅（32の倍数と仮定）
    def __init__(self, C, H, W):
        super(Discriminator, self).__init__()

        # 訓練データ量の不足を補うためのデータ拡張（Data Augmentation）処理
        # 詳しくは参考サイト（ https://qiita.com/T-STAR/items/e079da240d886bbb4ed0 など ）を参照
        self.preprocess = DiscriminatorAugmentation(H, W, p_hflip=0.5, p_vflip=0.4, p_rot=0.4) # 確率0.5で左右反転，確率0.4で上下反転，確率0.4で回転

        # ダウンサンプリング層1～5
        # カーネルサイズ4，ストライド幅2，パディング1の設定なので，これらを通すことにより特徴マップの縦幅・横幅がそれぞれ 1/2 になる
        self.down1 = myDownsamplingBlock(in_channels=C, out_channels=32)
        self.down2 = myDownsamplingBlock(in_channels=32, out_channels=64)
        self.down3 = myDownsamplingBlock(in_channels=64, out_channels=128)
        self.down4 = myDownsamplingBlock(in_channels=128, out_channels=256)
        self.down5 = myDownsamplingBlock(in_channels=256, out_channels=256)

        # 平坦化
        self.flat = nn.Flatten()

        # 全結合層1（spectral normalization を使用）
        # ダウンサンプリング層1～5を通すことにより特徴マップの縦幅・横幅は都合 1/32 になっているので，
        # 入力側のパーセプトロン数は 256*(H/32)*(W/32) = H*W/4
        self.fc1 = nn.utils.spectral_norm(nn.Linear(in_features=H*W//4, out_features=256))

        # 全結合層2
        # in_features の設定値については，次の Minibatch Discrimination に関するコメントを参照
        self.fc2 = nn.Linear(in_features=384, out_features=1)

        # Minibatch Discrimination: モード崩壊を回避するための技法の一つ
        # 詳しくは参考サイト（ http://www2.media.is.uec.ac.jp/column/20210507_ni など ）を参照
        # [使用方法] - in_features には直前の層の out_features と同じ値を設定する
        #            - out_features の設定は自由，ただし，in_features と out_features の和が直後の層の in_features に一致するようにする
        #            - このサンプルコードでは，md.in_features == fc1.out_features == 256
        #              かつ fc2.in_features == md.in_features + md.out_features == 256 + 128 == 384
        self.md = MinibatchDiscrimination(in_features=256, out_features=128)

    def forward(self, x):
        # 本来であれば，ディスクリミネータの出力が 0～1 の範囲となるよう，最終層の活性化関数として sigmoid を適用すべきであるが，
        # このサンプルコードでは損失関数側で sigmoid 適用することになるので, ここでは最終層で活性化関数を適用しない
        h = self.preprocess(x)
        h = self.down1(h)
        h = self.down2(h)
        h = self.down3(h)
        h = self.down4(h)
        h = self.down5(h)
        h = self.flat(h)
        h = F.relu(self.fc1(h))
        h = self.md(h) # Minibatch Discrimination
        z = self.fc2(h) # 上記の通り，最終層では活性化関数なし
        return z

##### 訓練データセットの読み込み

In [ ]:
from torchvision import transforms
from torch.utils.data import DataLoader
from mylib.data_io import CSVBasedDataset
from mylib.utility import save_datasets, load_datasets_from_file


# 前回の試行の続きを行いたい場合は True にする -> 再開モードになる
RESTART_MODE = False


# 再開モードの場合は，前回使用したデータセットをロードして使用する
if RESTART_MODE:
    train_dataset, _ = load_datasets_from_file(MODEL_DIR)
    if train_dataset is None:
        print('error: there is no checkpoint previously saved.')
        exit()
    train_size = len(train_dataset)

# そうでない場合は，データセットを読み込む
else:

    # CSVファイルを読み込み, 訓練データセットを用意
    # 今回は，全てのデータを学習用に回す
    train_dataset = CSVBasedDataset(
        filename = DATASET_CSV,
        items = [
            'File Path', # X
        ],
        dtypes = [
            'image', # Xの型
        ],
        dirname = DATA_DIR,
        img_transform = transforms.CenterCrop((H, W)), # 処理量を少しでも抑えるため，画像中央の H×W ピクセルの部分だけを対象とする
    )
    train_size = len(train_dataset)

    # データセット情報をファイルに保存
    save_datasets(MODEL_DIR, train_dataset)

# 訓練データをミニバッチに分けて使用するための「データローダ」を用意
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True)

##### 学習処理の実行
- GANの学習は一般に安定せず，最終的なモデルよりも学習途中のモデルの方が優れていることがよくあります
- このため，エポックごとにモデル保存処理を実行し，学習終了後，最良（と思われる）モデルをロードして利用することも多いです
- ただし，これを Paperspace Gradient などのクラウド環境で実行するとストレージ使用量の上限（Paperspace Gradient では 5GB）を超えてしまう可能性があるので，注意してください

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from mylib.loss_functions import GANLoss
from mylib.visualizers import LossVisualizer
from mylib.data_io import show_images, to_sigmoid_image, to_tanh_image, autosaved_model_name
from mylib.utility import save_checkpoint, load_checkpoint


# 前回の試行の続きを行いたい場合は True にする -> 再開モードになる
RESTART_MODE = False

# 何エポックに1回の割合で学習経過を表示するか（モデル保存処理もこれと同じ頻度で実行）
INTERVAL_FOR_SHOWING_PROGRESS = 10

# spectral normalization の使用によりディスクリミネータが弱体化するので，ジェネレータの更新回数を減らすことが望ましい（らしいが，実際にはなんとも言い難い）
# ここでは，ジェネレータを5回に1回の割合でしか更新しないことにする
N_DIS = 5 # この値を 1 にすれば，ジェネレータも毎回更新されるようになる


# エポック番号
INIT_EPOCH = 0 # 初期値
LAST_EPOCH = INIT_EPOCH + N_EPOCHS # 最終値

# ニューラルネットワークの作成
gen_model = Generator(C=C, H=H, W=W, N=N).to(DEVICE)
dis_model = Discriminator(C=C, H=H, W=W).to(DEVICE)

# 最適化アルゴリズムの指定（ここでは SGD でなく Adam を使用）
gen_optimizer = optim.Adam(gen_model.parameters(), lr=0.0002, betas=(0.5, 0.999))
dis_optimizer = optim.Adam(dis_model.parameters(), lr=0.0002, betas=(0.5, 0.999))

# 再開モードの場合は，前回チェックポイントから情報をロードして学習再開
if RESTART_MODE:
    INIT_EPOCH, LAST_EPOCH, gen_model, gen_optimizer = load_checkpoint(CHECKPOINT_EPOCH, CHECKPOINT_GEN_MODEL, CHECKPOINT_GEN_OPT, N_EPOCHS, gen_model, gen_optimizer)
    _, _, dis_model, dis_optimizer = load_checkpoint(CHECKPOINT_EPOCH, CHECKPOINT_DIS_MODEL, CHECKPOINT_DIS_OPT, N_EPOCHS, dis_model, dis_optimizer)
    print('')

# 損失関数
# Label Smooting（GANの学習を安定化させる技法の一つ）を採用
# 詳しくは参考サイト（ https://tatsy.github.io/programming-for-beginners/python/stabilize-gan-training/ など ）を参照
loss_func = GANLoss(label_smoothing=True)

# 検証の際に使用する乱数ベクトルを用意
Z_valid = torch.randn((BATCH_SIZE, N)).to(DEVICE)

# 損失関数値を記録する準備
loss_viz = LossVisualizer(['G loss', 'D loss'], init_epoch=INIT_EPOCH)

# 勾配降下法による繰り返し学習
for epoch in range(INIT_EPOCH, LAST_EPOCH):

    print('Epoch {0}:'.format(epoch + 1))

    # 学習
    gen_model.train()
    dis_model.train()
    sum_gen_loss = 0
    sum_dis_loss = 0
    n_iter = 1 # 1エポック内でのループ回数を記録する変数（ジェネレータの更新回数を制御するために使用）
    for X in tqdm(train_dataloader):
        for param in gen_model.parameters():
            param.grad = None
        for param in dis_model.parameters():
            param.grad = None
        Z = torch.randn((len(X), N)).to(DEVICE) # 乱数ベクトルを用意（通常の標準正規分布から作成）
        real = to_tanh_image(X).to(DEVICE) # Real画像を用意（to_tanh_image 関数を用い，画素値の範囲が -1〜1 となるように調整しておく）
        fake = gen_model(Z) # Fake画像を生成（2行上で用意した Z から生成）
        fake_cpy = fake.detach() # Fake画像のコピーを用意しておく
        ### ジェネレータの学習 ###
        if n_iter % N_DIS == 0:
            Y_fake = dis_model(fake) # Fake画像を識別
            gen_loss = loss_func.G_loss(Y_fake)
            gen_loss.backward()
            gen_optimizer.step()
            sum_gen_loss += float(gen_loss) * len(X)
        ### ディスクリミネータの学習 ###
        for param in dis_model.parameters():
            param.grad = None # ジェネレータの学習時の計算した勾配を一旦リセット
        Y_real = dis_model(real) # Real画像を識別
        Y_fake = dis_model(fake_cpy) # Fake画像を識別（コピー変数の方を使用）
        dis_loss = loss_func.D_loss(Y_fake, as_real=False) + loss_func.D_loss(Y_real, as_real=True)
        dis_loss.backward()
        dis_optimizer.step()
        sum_dis_loss += float(dis_loss) * len(X)
        n_iter += 1
    avg_gen_loss = sum_gen_loss * N_DIS / train_size
    avg_dis_loss = sum_dis_loss / train_size
    loss_viz.add_value('G loss', avg_gen_loss) # 訓練データに対する損失関数の値を記録
    loss_viz.add_value('D loss', avg_dis_loss) # 同上
    print('generator train loss = {0:.6f}'.format(avg_gen_loss))
    print('discriminator train loss = {0:.6f}'.format(avg_dis_loss))
    print('')

    # 検証（学習経過の表示，モデル自動保存）
    if epoch == 0 or (epoch + 1) % INTERVAL_FOR_SHOWING_PROGRESS == 0:
        gen_model.eval()
        dis_model.eval()
        if epoch == 0:
            real = to_sigmoid_image(real) # to_sigmoid_image 関数を用い，画素値が 0〜1 の範囲となるように調整する
            show_images(real.to('cpu').detach(), num=32, num_per_row=8, title='real images', save_fig=False, save_dir=MODEL_DIR) # Real画像の例を表示（最初のエポックのみ）
        with torch.inference_mode():
            fake = gen_model(Z_valid) # 事前に用意しておいた検証用乱数からFake画像を生成
            #fake = gen_model(torch.randn((BATCH_SIZE, N)).to(DEVICE)) # エポックごとに異なる乱数を使用する場合はこのようにする
        fake = to_sigmoid_image(fake) # to_sigmoid_image 関数を用い，画素値が 0〜1 の範囲となるように調整する
        show_images(fake.to('cpu').detach(), num=32, num_per_row=8, title='epoch {0}'.format(epoch + 1), save_fig=False, save_dir=MODEL_DIR) # 現在のジェネレータによるFake画像の例を表示
        torch.save(gen_model.state_dict(), autosaved_model_name(MODEL_FILE_G, epoch + 1)) # 学習途中のモデルを保存したい場合はこのようにする

    # 現在の学習状態を一時ファイル（チェックポイント）に保存
    save_checkpoint(CHECKPOINT_EPOCH, CHECKPOINT_GEN_MODEL, CHECKPOINT_GEN_OPT, epoch+1, gen_model, gen_optimizer)
    save_checkpoint(CHECKPOINT_EPOCH, CHECKPOINT_DIS_MODEL, CHECKPOINT_DIS_OPT, epoch+1, dis_model, dis_optimizer)

# 学習結果のニューラルネットワークモデルをファイルに保存
gen_model = gen_model.to('cpu')
dis_model = dis_model.to('cpu')
torch.save(gen_model.state_dict(), MODEL_FILE_G)
#torch.save(dis_model.state_dict(), MODEL_FILE_D) # ディスクリミネータも保存したい場合はこのようにする

# 損失関数の記録をファイルに保存
loss_viz.save(v_file=os.path.join(MODEL_DIR, 'loss_graph.png'), h_file=os.path.join(MODEL_DIR, 'loss_history.csv'))

##### 学習済みニューラルネットワークモデルのロード

In [ ]:
import torch


# ニューラルネットワークモデルとその学習済みパラメータをファイルからロード
gen_model = Generator(C=C, H=H, W=W, N=N).to(DEVICE)
gen_model.load_state_dict(torch.load(MODEL_FILE_G)) # 最終モデルをロードする場合
#gen_model.load_state_dict(torch.load(autosaved_model_name(MODEL_FILE_G, 80))) # 例えば80エポック目のモデルをロードしたい場合は，このようにする

##### テスト処理（正規分布に従ってランダムサンプリングした乱数をデコーダに通して画像を生成）

In [ ]:
import torch
from mylib.data_io import show_images, to_sigmoid_image


gen_model = gen_model.to(DEVICE)
gen_model.eval()

# 生成する画像の枚数
n_gen = 32

# 標準正規分布 N(0,1) に従って適当に乱数ベクトルを作成
Z = torch.randn((n_gen, N)).to(DEVICE)

# 乱数ベクトルをデコーダに入力し，その結果を表示
with torch.inference_mode():
    Y = gen_model(Z)
    Y = to_sigmoid_image(Y)
    show_images(Y.to('cpu').detach(), num=n_gen, num_per_row=8, title='GAN_sample_generated', save_fig=True)